In [ ]:
# !pip install opencv-python-headless==4.8.0.74 openmim==0.1.5 timm==0.5.4 torch==1.11.0 torchvision==0.12.0
!pip install opencv-python-headless openmim timm torch torchvision


In [ ]:
!mim install mmcv-full


/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
installing mmcv-full from wheel.
Looking in links: https://download.openmmlab.com/mmcv/dist/cpu/torch1.11.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 7.0 MB/s eta 0:00:00


In [ ]:
# !mim install mmcv==2.0.0

Looking in links: https://download.openmmlab.com/mmcv/dist/cu102/torch1.11.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.7/451.7 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 762.6/762.6 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached mmdet-2.24.1.tar.gz (762 kB)
  Preparing metadata (setup.py) ... done
  Using cached mmdet-2.24.1.tar.gz (762 kB)
  Preparing metadata (setup.py) ... done
  Using cached mmdet-2.24.1.tar.gz (762 kB)
  Preparing metadata (setup.py) ... done
  Using cached mmdet-2.24.1.tar.gz (762 kB)
  Preparing metadata (setup.py) ... done
  Using cached mmdet-2.24.1.tar.gz (762 kB)
  Preparing metadata (setup.py) ... done
  Using cached mmdet-2.24.1.tar.gz (762 k

In [ ]:
!pip install mmdet mmpose mmengine

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.7/451.7 kB 7.9 MB/s eta 0:00:00


In [ ]:
# !git clone https://github.com/ViTAE-Transformer/ViTPose.git

In [ ]:
# !mim install mmdet==2.24.1 mmpose
# !mim install mmdet
#↑これをやるとmmcv==2.1.0のインストールを始めて、やたら時間がかかる。その後エラー
#先にmmcvをインストールする必要があるのかも
!pip install mmdet==2.24.1 mmpose

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 4.0 MB/s eta 0:00:00
  Attempting uninstall: mmdet
    Found existing installation: mmdet 3.3.0
    Uninstalling mmdet-3.3.0:
      Successfully uninstalled mmdet-3.3.0


In [ ]:
!pip install yapf==0.31.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.7/185.7 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: yapf
    Found existing installation: yapf 0.40.2
    Uninstalling yapf-0.40.2:
      Successfully uninstalled yapf-0.40.2


In [ ]:
!mim download mmdet --config yolox_tiny_8x8_300e_coco --dest /content

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/usr/local/lib/python3.10/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
processing yolox_tiny_8x8_300e_coco...
  [####################################]  100%          
Successfully downloaded yolox_tiny_8x8_300e_coco_20211124_171234-b4047906.

In [ ]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from mmdet.apis import inference_detector, init_detector
import urllib.request

# 物体検出モデルの初期化
class DetModel:

    def __init__(self):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.model = self._load_model()

    def _load_model(self):
        config_path = "/content/yolox_tiny_8x8_300e_coco.py"
        model_path = "/content/yolox_tiny_8x8_300e_coco_20211124_171234-b4047906.pth"

        return init_detector(config_path, model_path, device=self.device)

    def detect(self, image: np.ndarray):
        image = image[:, :, ::-1]  # RGB -> BGR
        out = inference_detector(self.model, image)
        return out

    def visualize_detection_results(self, image: np.ndarray, detection_results: list, score_threshold: float = 0.3):
        person_det = [detection_results[0]] + [np.array([]).reshape(0, 5)] * 79
        image = image[:, :, ::-1]  # RGB -> BGR
        vis = self.model.show_result(
            image, person_det, score_thr=score_threshold, bbox_color=None, text_color=(200, 200, 200), mask_color=None
        )
        return vis[:, :, ::-1]  # BGR -> RGB



det_model = DetModel()

AssertionError: MMCV==1.7.2 is used but incompatible. Please install mmcv>=2.0.0rc4, <2.2.0.

In [ ]:
# !mim install 'mmpretrain>=1.0.0'

In [ ]:
# !git clone https://github.com/open-mmlab/mmpose.git

In [ ]:
# !mim install mmpose

In [ ]:
# !mim install mmengine

In [ ]:
# !pip install -v mmcv-full>=2.0.0rc4

In [ ]:
from mmpose.apis import inference_top_down_pose_model, init_pose_model, process_mmdet_results, vis_pose_result
# 姿勢推定モデルの初期化
class PoseModel:
    def __init__(self):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.model = self._load_model()

    def _load_model(self):
        ckpt_path = "https://download.openmmlab.com/mmpose/v1/body_2d_keypoint/topdown_heatmap/coco/td-hm_ViTPose-small_8xb64-210e_coco-256x192-62d7a712_20230314.pth"
        config_path = "/content/mmpose/configs/body_2d_keypoint/topdown_heatmap/coco/td-hm_ViTPose-base-simple_8xb64-210e_coco-256x192.py"

        model = init_pose_model(config_path, ckpt_path, device=self.device)
        return model

    def predict_pose(self, image: np.ndarray, det_results: list, box_score_threshold: float = 0.5):
        image = image[:, :, ::-1]  # RGB -> BGR
        person_results = process_mmdet_results(det_results, 1)
        out, _ = inference_top_down_pose_model(
            self.model, image, person_results=person_results, bbox_thr=box_score_threshold, format="xyxy"
        )
        return out

    def visualize_pose_results(self, image: np.ndarray, pose_results: list, kpt_score_threshold: float = 0.3, vis_dot_radius: int = 4, vis_line_thickness: int = 1):
        image = image[:, :, ::-1]  # RGB -> BGR
        vis = vis_pose_result(
            self.model,
            image,
            pose_results,
            kpt_score_thr=kpt_score_threshold,
            radius=vis_dot_radius,
            thickness=vis_line_thickness,
        )
        return vis[:, :, ::-1]  # BGR -> RGB

pose_model = PoseModel()

AssertionError: MMCV==1.5.0 is used but incompatible. Please install mmcv>=2.0.0rc4, <=3.0.0.

In [ ]:
# 画像のアップロード
uploaded = files.upload()

for filename in uploaded.keys():
    image_path = filename

# 画像の読み込み
image = plt.imread(image_path)

In [ ]:
# 物体検出
det_results = det_model.detect(image)
det_visualization = det_model.visualize_detection_results(image, det_results)

# 検出結果の表示
plt.figure(figsize=(10, 10))
plt.imshow(det_visualization)
plt.title("Detection Results")
plt.axis("off")
plt.show()

In [ ]:
# 姿勢推定
pose_results = pose_model.predict_pose(image, det_results)
pose_visualization = pose_model.visualize_pose_results(image, pose_results)

# 姿勢推定結果の表示
plt.figure(figsize=(10, 10))
plt.imshow(pose_visualization)
plt.title("Pose Estimation Results")
plt.axis("off")
plt.show()